# `pctheory` basics

`pctheory` is a Python library for working with atonal theory. The most fundamental classes in the `pctheory` library are `pctheory.pitch.Pitch`, `pctheory.pitch.PitchClass`, `pctheory.transformations.OTO`, and `pctheory.transformations.UTO`. This basic guide to `pctheory` shows you how to use these classes.

`pctheory` uses standard Python data structures to represent sets (the Python `set`) and rows (the Python `list`).

## Basic structures
The example below shows how to make pitches, pitch-classes, pitch-class sets, and an ordered pitch-class hexachord (half of a twelve-tone row). You'll notice that `pctheory` represents pitch-class 10 as `A` and pitch-class 11 as `B`.

In [1]:
from pctheory import pitch, pcseg, pcset, transformations

# A pitch. 0 is C4 (corresponding to 60 in MIDI)
p1 = pitch.Pitch(14)

# A pitch-class
pc1 = pitch.PitchClass(4)

# A pitch-class set
set1 = {pitch.PitchClass(5), pitch.PitchClass(3), pitch.PitchClass(10)}

# An easier way to make the same pitch-class set
set1 = pcset.make_pcset12(5, 3, 10)

# An ordered hexachord
permutation1 = [pitch.PitchClass(3), pitch.PitchClass(5), pitch.PitchClass(2), pitch.PitchClass(0), pitch.PitchClass(9), pitch.PitchClass(11)]

# An easier way to make the same ordered hexachord
permutation1 = pcseg.make_pcseg12(3, 5, 2, 0, 9, 11)

print(p1)
print(pc1)
print(set1)
print(permutation1)

14
4
{5, A, 3}
[3, 5, 2, 0, 9, B]


## Operators
The `OTO` and `UTO` classes in the `transformations` module are for working with `PitchClass` objects and collections of `PitchClass`es.

`OTO` stands for "ordered tone operator" and corresponds to a "row operator" or "RO" in Robert Morris's terminology.
`UTO` stands for "unordered tone operator" and corresponds to a "twelve tone operator" or "TTO" in Robert Morris's terminology. These terms have been renamed here because `pctheory` allows working with 24-tone microtonal structures.

If you are transforming a single pitch-class or a pitch-class set, you should use `UTO`. The `UTO` class lets you perform a multiplication operation, followed by a transposition operation. (Note that inversion is the same as multiplying by 11, mod 12).

In [6]:
# T5I (invert, then transpose by 5 semitones)
op1 = transformations.UTO(5, 11)

# But it's easier to just provide the transformation string
op1 = transformations.UTO("T5I")

# The same operator can transform a pitch-class or a pitch-class set
print(op1(pc1))
print(op1(set1))

1
{7, 0, 2}


If you are transforming a twelve-tone row (or other ordered permutation of pitch-classes), you should use `OTO`. This operator lets you perform a multiplication operation, followed by a transposition operation. It also allows you to perform a retrograde operation.

In [5]:
# T4R (multiply by 1, which does nothing, transpose by 4, and retrograde)
op2 = transformations.OTO(4, True, 1)

# T4I (multiply by 11, which inverts, transpose by 4, and do not retrograde)
op3 = transformations.OTO(4, False, 11)

# An easier way: just provide the transformation string
op2 = transformations.OTO("T4R")
op3 = transformations.OTO("T4I")

print(op2(permutation1))
print(op3(permutation1))

[3, 1, 4, 6, 9, 7]
[1, B, 2, 4, 7, 5]


If you want to combine several transformations into one simpler transformation, you can use the "left multiply" feature in the `transformations` module. This is because any operator of the form $T_{n_1}M_{m_1}$ where $m \in \{1, 5, 7, 11\}$ can be combined with any other operator of the form $T_{n_2}M_{m_2}$ to make an operator of the form $T_{n_3}M_{m_3}$ using the formula $T_{n_1}M_{m_1}T_{n_2}M_{m_2} = T_{n_1m_2 + n_2}M_{m_1m_2}$.

In [7]:
op4 = transformations.UTO(5, 1)
op5 = transformations.UTO(2, 11)
op6 = transformations.UTO(9, 1)

combined = transformations.left_multiply_utos(op4, op5, op6)
print(combined)

T10M11


It is often convenient to have all of the standard operators in a dictionary, so they can be used any time:

In [8]:
tto = transformations.get_utos12()  # gets all of the TTOs
print(tto)
print(tto["T5I"](set1))
print(tto["T11"](set1))
print(tto["T3M"](set1))
print(tto["T4MI"](set1))

{'T0': T0, 'T0M': T0M5, 'T0MI': T0M7, 'T0I': T0M11, 'T1': T1, 'T1M': T1M5, 'T1MI': T1M7, 'T1I': T1M11, 'T2': T2, 'T2M': T2M5, 'T2MI': T2M7, 'T2I': T2M11, 'T3': T3, 'T3M': T3M5, 'T3MI': T3M7, 'T3I': T3M11, 'T4': T4, 'T4M': T4M5, 'T4MI': T4M7, 'T4I': T4M11, 'T5': T5, 'T5M': T5M5, 'T5MI': T5M7, 'T5I': T5M11, 'T6': T6, 'T6M': T6M5, 'T6MI': T6M7, 'T6I': T6M11, 'T7': T7, 'T7M': T7M5, 'T7MI': T7M7, 'T7I': T7M11, 'T8': T8, 'T8M': T8M5, 'T8MI': T8M7, 'T8I': T8M11, 'T9': T9, 'T9M': T9M5, 'T9MI': T9M7, 'T9I': T9M11, 'T10': T10, 'T10M': T10M5, 'T10MI': T10M7, 'T10I': T10M11, 'T11': T11, 'T11M': T11M5, 'T11MI': T11M7, 'T11I': T11M11}
{7, 0, 2}
{4, 9, 2}
{5, 4, 6}
{1, 3, 2}


The `UTO` and `OTO` classes are not the only way to transform pitch-class sets and rows. You can also use these functions:
- `pctheory.pcseg.invert`
- `pctheory.pcseg.multiply`
- `pctheory.pcseg.transpose`
- `pctheory.pcset.invert`
- `pctheory.pcset.multiply`
- `pctheory.pcset.transpose`

However, it is often simpler to use an operator class to perform a given transformation.

## Mathematical foundation of operators
The `OTO` and `UTO` operators use addition and multiplication mod 12.

### Transposition
Transposition corresponds to addition. $T_5(9) = (9 + 5) \textrm{ mod } 12 = 2$.

### Multiplication
There are four multipliers that work for multiplication: 1, 5, 7, and 11. The other multipliers aren't one-to-one operations (they can't produce all twelve pitch-classes), which makes them unsuitable for most uses.

Multiplication by 1 is an *identity* operation - it makes no change. $M_1(9) = (9 \cdot 1) \textrm{ mod } 12 = 9$.

Multiplication by 11 is equivalent to inversion. $M_{11}(9) = I(9) = (9 \cdot 11) \textrm{ mod } 12 = 3$.

Multiplication by 5 sometimes maps into the same set-class, and sometimes into a different set-class. The same is true of multiplication by 7. It turns out that multiplication by 7 is the same as multiplication by 5, then multiplication by 11. For this reason, $M_{11}$ (inversion) is often denoted as $I$, $M_5$ is often denoted simply as $M$, and $M_7$ is often denoted as $MI$.

### Why multiply first, then transpose?
You don't have to multiply first - you can perform the multiplication second. However, this will always be the same as some other operation that performed the multiplication first, then transposition second. The index of transposition may be different, but the result will be the same. For simplicity, the operators in `pctheory` multiply first.

## Conclusion
This tutorial covers the basics of working with `pctheory`. For more specific information, check out the tutorials on pitch class sets and rows.